In [ ]:
import os
import pandas as pd
import numpy as np
from timm import create_model
from utils import seed_everything
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F

# Paths
root = '/media/latlab/MR/projects/kaggle-hms'
data_dir = os.path.join(root, 'data')
results_dir = os.path.join(root, 'results')
train_eeg_dir = os.path.join(data_dir, 'train_eegs')
train_spectrogram_dir = os.path.join(data_dir, 'train_spectrograms')

# Seed
seed_everything(42)

# Load data
df = pd.read_csv(os.path.join(data_dir, 'train.csv'))
spec_data = np.load(os.path.join(data_dir, 'spec_data.npy'), allow_pickle=True).item()

In [ ]:
from model.metric import kl_divergence
from ext.kaggle_kl_div.kaggle_kl_div import score
a = pd.DataFrame(np.array([[0.8, 0.0, 0.2, 0.0], [0.8, 0.0, 0.2, 0.0]]))
a['id'] = np.arange(len(a))
b = pd.DataFrame(np.array([[0.2, 0.0, 0.8, 0.0], [0.99, 0.01, 0.0, 0.0]]))
b['id'] = np.arange(len(b))
print(score(a, b, row_id_column_name=None))
print(kl_divergence(a, b))

In [ ]:
img_orig = spec_data[353733][:300,:]
img_orig.shape

In [ ]:
X = np.zeros((1,128,256,8),dtype='float32')
for k in range(4):
    img = img_orig[:, k*100:(k+1)*100].T

    # LOG TRANSFORM SPECTROGRAM
    img = np.clip(img,np.exp(-4),np.exp(8))
    img = np.log(img)

    # STANDARDIZE PER IMAGE
    ep = 1e-6
    m = np.nanmean(img.flatten())
    s = np.nanstd(img.flatten())
    img = (img-m)/(s+ep)
    img = np.nan_to_num(img, nan=0.0)

    # CROP TO 256 TIME STEPS
    X[0, 14:-14,:,k] = img[:,22:-22] / 2.0
plt.imshow(X[0,:,:,0])

In [ ]:
img = img_orig.T

# LOG TRANSFORM SPECTROGRAM
img = np.clip(img,np.exp(-4),np.exp(8))
img = np.log(img)

# STANDARDIZE PER IMAGE
ep = 1e-6
m = np.nanmean(img.flatten())
s = np.nanstd(img.flatten())
img = (img-m)/(s+ep)
img = np.nan_to_num(img, nan=0.0)
    
plt.imshow(img)

In [ ]:
# img = torch.from_numpy(img)
torch.stack([img, img, img], dim=1).shape

In [ ]:
x1 = [torch.tensor(X[:,:,:,i:i+1]) for i in range(4)]
plt.imshow(torch.cat(x1, dim=1).numpy()[0,:,:,0])